<a href="https://colab.research.google.com/github/arvindnswamy/RemoteSensing/blob/development/SupervisedClassificationWithPolygons_MyOwnCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Python example based on a GEE tutorial available at https://developers.google.com/earth-engine/classification
#This is the example that uses libsvm classifier.
#Important thing is that the image has to be a float. IT DOES NOT WORK IF THE IMAGE IS np.uint16, AS MOST LANDST
#IMAGES ARE. HENCE, I HAD TO CHANGE sat from sat = 'LANDSAT/LC08/C01/T1' to sat = 'LANDSAT/LC08/C01/T1_TOA'

import ee
from IPython.display import Image

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=XL7GhG_m993eAYT0CUpXFTsdnQgLoBJffYzttXKsn2s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AEmPRsza_1K4PLUkTVg_Li3h-x56aXP_un2DEw_wfUOr84He0YkBQ8

Successfully saved authorization token.


In [4]:
ee.Initialize()

def get_images(path_list, row_list, satellite, start_date, end_date, max_cloud_percentage):
  coll = ee.ImageCollection(satellite).filterDate(start_date, end_date)\
            .filter(ee.Filter.inList('WRS_PATH', path_list))\
            .filter(ee.Filter.inList('WRS_ROW', row_list))\
            .filter(ee.Filter.lt('CLOUD_COVER', max_cloud_percentage))
  image_ids = list(map(lambda x: x['id'], coll.getInfo()['features']))

  images = list(map(lambda x: ee.Image(x), image_ids))

  return images

In [5]:
p = [14]
r = [32]
sat = 'LANDSAT/LC08/C01/T1_TOA'
sd = '2013-05-01'
ed = '2020-05-01'
cc= 10

image_list_nyc = get_images(p, r, sat, sd, ed, cc)

image_list_nyc

In [6]:
'''
Functions needed to display images and features/featurecollections are given here. 
These functions are all from other sources. Mostly from google tutorials.
'''
!pip install geojson pygeoj
import geojson
import json
import pygeoj
import numpy as np


import folium


# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

  Created wheel for pygeoj: filename=PyGeoj-1.0.0-cp36-none-any.whl size=10348 sha256=a33851171be54c33ececffc117aa944e475dce2dea09c47c0d6c0823f986a58e
  Stored in directory: /root/.cache/pip/wheels/7d/7c/8e/3c82fc15ee9f04d8df3e59817d2ee5499a8413341d6476c7d8
Successfully built pygeoj


In [9]:
'''
In this cell, I show how you can take GeoJSON files and convert them into Google Earth Engine features and 
featurecollections. You might have created the GeoJSON files by some other means - one of them is
to use Google Earth to draw a polygon, save it as a kml file, and use ogr2ogr to conert  it to a GeoJSON file. I
created 4 such files, titled 'Hudson01.json', 'Hudson02.json', 'Bronx01.json', and 'Astoria01.json'. 

For those from NYC, or if you have a knowledge of the geography of NYC, you'd have guess  that 'Hudson01.json', and
'Hudson02.json' are regions in water, and that 'Bronx01.json', and 'Astoria01.json' correspond to land (not water).
Hence, I have created two lists jsonfiles and classalloc. landcover = 1 corresponds to water, and landcover = 0 to 
'not water'. 
'''

jsonfiles = ['Hudson01', 'Hudson02', 'Bronx01', 'Astoria01'];
classalloc = [1, 1, 0, 0];

'''
These dictionaries are to store intermediate objects before we get to features and feature collections. First,
we extract the data from the GeoJSON file. Then we extract the coordinates as a list. The list is converted into
a earth engine polygon. Finally, the polygon is converted into a feature. The feature has two attributes - 'name'
and 'class'. Both are taken from corresponding values of jsonfiles and classalloc.

We need to merge all these featurecollections into one final featurecollection that can be used for training.
That is what is achieved by the lines:

fc = ee.FeatureCollection([])
for x in randomPts_dict.keys():
    print(x)
    fc = fc.merge(randomPts_dict[x])

    
'''
coords_dict = {}
ee_dict = {}
features_dict = {}

n = 0
for jsonfile in jsonfiles:
    jsonfilepath = 'sample_data/' + jsonfile +'.json'
    with open(jsonfilepath) as f:
        data = geojson.load(f)
    
    #creating a dictionary of coordinates
    coords_dict[jsonfile + 'coords'] = np.array(data['features'][0]['geometry']['coordinates'][0])[:,0:2].tolist()
    
    #creating a polygon from coordinate list
    ee_dict[jsonfile + 'ee'] = ee.Geometry.Polygon(coords_dict[jsonfile + 'coords'])
    
    
    #randomPts_dict[jsonfile + 'rdnmPts'] = ee.FeatureCollection.randomPoints(ee_dict[jsonfile + 'ee'], 100)
    features_dict[jsonfile + 'feature'] = ee.Feature(ee_dict[jsonfile + 'ee'], {'landcover': classalloc[n]})

    n = n+1

    
'''
The individual features are combined as shown below to create a feature collection.

You can get some information about the features in the collection using commands as shown below.
'''
nycFC = ee.FeatureCollection(list(features_dict.values()))


fc = ee.FeatureCollection([])
for x in features_dict.keys():
    #print(x) #I had used this to make sure the loop was okay
    fc = fc.merge(ee.FeatureCollection([features_dict[x]]))
#fc = ee.FeatureCollection([features_dict['Hudson01feature'], features_dict['Hudson02feature'], features_dict['Bronx01feature'], features_dict['Astoria01feature']])


In [10]:
nycsite = ee.Geometry.Rectangle([-74.04, 40.69, -73.82, 40.94]);

image0 = image_list_nyc[0] #one image from the list of images

#l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1');
#image0 = ee.Algorithms.Landsat.simpleComposite(collection = l8.filterDate('2018-03-01', '2018-04-30'), asFloat = True)
'''
when I used image0 = image_list_nyc[0] or image0 = image_list_nyc[0].float(), I got errors.
When I used image0 = ee.Algorithms.Landsat.simpleComposite(collection = l8.filterDate('2018-03-01', '2018-04-30'), asFloat = True),
I got better answers.
'''
parameters = {'min': 0.0,
              'max': 1.0,
              'dimensions': 600,
              'bands': ['B4', 'B3', 'B2']}

Image(url = image0.clip(nycsite).getThumbUrl(parameters))

In [11]:
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']
#bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

training = image0.sampleRegions(collection =  fc, properties = ['landcover'], scale = 30.0)
print(training.size().getInfo())

21898


In [12]:
classifier = ee.Classifier.libsvm(kernelType = 'RBF', gamma=0.5, cost = 1.0)

In [13]:
trained = classifier.train(training, 'landcover', bands)

In [14]:
classified = image0.classify(trained)

In [15]:
my_map = folium.Map(location=[ 40.8, -73.8], zoom_start=9, height=500)
my_map.add_ee_layer(classified, {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'water')
#display(my_map)

In [16]:
display(my_map)

In [17]:
type(classified)

ee.image.Image

In [21]:
type(features_dict['Astoria01feature'])

ee.feature.Feature

In [24]:
type(image0.float())

ee.image.Image

In [72]:
training.toList(100).getInfo()

[{'geometry': None,
  'id': '0_0',
  'properties': {'B1': 0.14788319170475006,
   'B10': 294.01617431640625,
   'B11': 292.6298828125,
   'B2': 0.13301372528076172,
   'B3': 0.12088567763566971,
   'B4': 0.10728823393583298,
   'B5': 0.06925924867391586,
   'B6': 0.046582214534282684,
   'B7': 0.037195589393377304,
   'B8': 0.1151396632194519,
   'B9': 0.0006360051920637488,
   'BQA': 2720,
   'landcover': 1},
  'type': 'Feature'},
 {'geometry': None,
  'id': '0_1',
  'properties': {'B1': 0.14621640741825104,
   'B10': 293.94952392578125,
   'B11': 292.600830078125,
   'B2': 0.13108375668525696,
   'B3': 0.11998648941516876,
   'B4': 0.10487578064203262,
   'B5': 0.06570636481046677,
   'B6': 0.044937361031770706,
   'B7': 0.03539721667766571,
   'B8': 0.11428433656692505,
   'B9': 0.000899181526619941,
   'BQA': 2720,
   'landcover': 1},
  'type': 'Feature'},
 {'geometry': None,
  'id': '0_2',
  'properties': {'B1': 0.1467866152524948,
   'B10': 293.8877868652344,
   'B11': 292.571777

In [56]:
parameters = {'min': 0.0,
              'max': 1.0,
              'dimensions': 600,
              'palette': ['white', 'blue']}

Image(url = classified.clip(nycsite).getThumbUrl(parameters))

In [57]:
classified.toArray().getInfo()

{'bands': [{'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 393285, 0, -30, 4575615],
   'data_type': {'dimensions': 1,
    'max': 2147483647,
    'min': -2147483648,
    'precision': 'int',
    'type': 'PixelType'},
   'dimensions': [7641, 7431],
   'id': 'array'}],
 'properties': {'system:footprint': {'coordinates': [[-73.55972856517639,
     40.9210458395994],
    [-73.65884701015169, 40.94040209951729],
    [-74.39099607563138, 41.08036506267204],
    [-75.53752851259908, 41.28907332531779],
    [-75.75831185535647, 41.32779956407564],
    [-75.7591722695154, 41.32647346866513],
    [-76.00264695366093, 40.53106663014035],
    [-76.24236827999674, 39.729154487819734],
    [-76.23573771307694, 39.72794945032572],
    [-75.79848047772803, 39.64906773852695],
    [-74.08398344632296, 39.3220193126467],
    [-74.06195535083428, 39.388950588916195],
    [-73.98986911699056, 39.60737895551066],
    [-73.95692468542457, 39.70682915736417],
    [-73.90842792248759, 39.85274831459669],
   